In [21]:
import json

In [3]:
import base64
from algosdk.v2client import algod
from algosdk import account, mnemonic
from algosdk.future.transaction import AssetConfigTxn, AssetTransferTxn, AssetFreezeTxn
from algosdk.future.transaction import *

In [7]:
from dotenv import load_dotenv

In [8]:
import sys, os

sys.path.append(os.path.abspath(os.path.join("../..")))
sys.path.append(os.path.abspath(os.path.join("../scripts")))

In [23]:
import functions_ASA as util

# Accounts

In [10]:
load_dotenv()
mnemonic1 = os.getenv('PreconfigPassphrase1')
mnemonic2 = os.getenv('PreconfigPassphrase2')
mnemonic3 = os.getenv('PreconfigPassphrase3')

In [11]:
# For ease of reference, add account public and private keys to
# an accounts dict.
accounts = {}
counter = 1
for m in [mnemonic1, mnemonic2, mnemonic3]:
    accounts[counter] = {}
    accounts[counter]['pk'] = mnemonic.to_public_key(m)
    accounts[counter]['sk'] = mnemonic.to_private_key(m)
    counter += 1

# Connect to Client

In [12]:
# Specify the node address and token.

algod_address = "http://localhost:4001"
algod_token = "aaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaa"

# Initialize an algod client
algod_client = algod.AlgodClient(algod_token=algod_token, algod_address=algod_address)

In [13]:
print("Account 1 address: {}".format(accounts[1]['pk']))
print("Account 2 address: {}".format(accounts[2]['pk']))
print("Account 3 address: {}".format(accounts[3]['pk']))

Account 1 address: BOBMBO72PB2AM4RBSHB7B3ZZML2AW4SDIUFKJ3YLCPSDKNO3FOCRMN7H2A
Account 2 address: WI6KG2FRR6FY7RSZH4AO4WDBY57R5IA74RX22EDQA4C6AOGVXAUAPUYWRU
Account 3 address: 2JZRZNDWUIEHMH3G7JMGI6JHD64FSC4FQ53AVBVSQABWBRLMTQMJQ6KF2U


# Create Asset (Minting)

* Account 1 creates an asset called latinum and
* sets Account 2 as the manager, reserve, freeze, and clawback address.

In [14]:
# Get network params for transactions before every transaction.
params = algod_client.suggested_params()

# Account 1 creates an asset called latinum and
# sets Account 2 as the manager, reserve, freeze, and clawback address.
# Asset Creation transaction

txn = AssetConfigTxn(
    sender=accounts[1]['pk'],
    sp=params,
    total=1000,
    default_frozen=False,
    unit_name="LATINUM",
    asset_name="latinum",
    manager=accounts[2]['pk'],
    reserve=accounts[2]['pk'],
    freeze=accounts[2]['pk'],
    clawback=accounts[2]['pk'],
    url="https://path/to/my/asset/details", 
    decimals=0)
    
# Sign with secret key of creator
stxn = txn.sign(accounts[1]['sk'])


In [15]:
# Send the transaction to the network and retrieve the txid.
try:
    txid = algod_client.send_transaction(stxn)
    print("Signed transaction with txID: {}".format(txid))
    # Wait for the transaction to be confirmed
    confirmed_txn = wait_for_confirmation(algod_client, txid, 4)  
    print("TXID: ", txid)
    print("Result confirmed in round: {}".format(confirmed_txn['confirmed-round']))
   
except Exception as err:
    print(err)
# Retrieve the asset ID of the newly created asset by first
# ensuring that the creation transaction was confirmed,
# then grabbing the asset id from the transaction.

Signed transaction with txID: H2J5SYNOQSX3DTCRK4XV3FU2VXTAS6DPU74A2MCGWJOMLBJJA5TA
TXID:  H2J5SYNOQSX3DTCRK4XV3FU2VXTAS6DPU74A2MCGWJOMLBJJA5TA
Result confirmed in round: 2


In [16]:
print("Transaction information: {}".format(
    json.dumps(confirmed_txn, indent=4)))

Transaction information: {
    "asset-index": 2,
    "confirmed-round": 2,
    "pool-error": "",
    "txn": {
        "sig": "r6EW0+6i0kAV8Vf5iPJg8lndHIpCEawiZd4BYe355n483BYYAhEBjC1QlQGmcjS4Yg4MxNB3ZJQ7zhYZj1C/BQ==",
        "txn": {
            "apar": {
                "an": "latinum",
                "au": "https://path/to/my/asset/details",
                "c": "WI6KG2FRR6FY7RSZH4AO4WDBY57R5IA74RX22EDQA4C6AOGVXAUAPUYWRU",
                "f": "WI6KG2FRR6FY7RSZH4AO4WDBY57R5IA74RX22EDQA4C6AOGVXAUAPUYWRU",
                "m": "WI6KG2FRR6FY7RSZH4AO4WDBY57R5IA74RX22EDQA4C6AOGVXAUAPUYWRU",
                "r": "WI6KG2FRR6FY7RSZH4AO4WDBY57R5IA74RX22EDQA4C6AOGVXAUAPUYWRU",
                "t": 1000,
                "un": "LATINUM"
            },
            "fee": 1000,
            "fv": 1,
            "gen": "sandnet-v1",
            "gh": "1Av0Ow3cvZTlmk8Nl+rYpdaYl8sFmSD4gie6oNNsKOA=",
            "lv": 1001,
            "snd": "BOBMBO72PB2AM4RBSHB7B3ZZML2AW4SDIUFKJ3YLCPSDKNO3FOCRMN7H2A

In [17]:
try:
    # Pull account info for the creator
    # account_info = algod_client.account_info(accounts[1]['pk'])
    # get asset_id from tx
    # Get the new asset's information from the creator account
    ptx = algod_client.pending_transaction_info(txid)
    asset_id = ptx["asset-index"]
    util.print_created_asset(algod_client, accounts[1]['pk'], asset_id)
    util.print_asset_holding(algod_client, accounts[1]['pk'], asset_id)
except Exception as e:
    print(e)

Asset ID: 2
name 'json' is not defined


# Change the Manager

The current manager(Account 2) issues an asset configuration transaction that assigns Account 1 as the new manager.

In [18]:
# Keep reserve, freeze, and clawback address same as before, i.e. account 2
params = algod_client.suggested_params()
# comment these two lines if you want to use suggested params
# params.fee = 1000
# params.flat_fee = True

# asset_id = 328952;

txn = AssetConfigTxn(
    sender=accounts[2]['pk'],
    sp=params,
    index=asset_id, 
    manager=accounts[1]['pk'],
    reserve=accounts[2]['pk'],
    freeze=accounts[2]['pk'],
    clawback=accounts[2]['pk'])
# sign by the current manager - Account 2
stxn = txn.sign(accounts[2]['sk'])
# txid = algod_client.send_transaction(stxn)
# print(txid)

In [19]:
# Wait for the transaction to be confirmed
# Send the transaction to the network and retrieve the txid.
try:
    txid = algod_client.send_transaction(stxn)
    print("Signed transaction with txID: {}".format(txid))
    # Wait for the transaction to be confirmed
    confirmed_txn = wait_for_confirmation(algod_client, txid, 4) 
    print("TXID: ", txid)
    print("Result confirmed in round: {}".format(confirmed_txn['confirmed-round']))
 
except Exception as err:
    print(err)

Signed transaction with txID: AWHTAPFRJAUORJW4UFFUC2GZ7ADING4CCIJEZXSS6DPKD5YE5X6Q
TXID:  AWHTAPFRJAUORJW4UFFUC2GZ7ADING4CCIJEZXSS6DPKD5YE5X6Q
Result confirmed in round: 3


In [24]:
# Check asset info to view change in management. manager should now be account 1
util.print_created_asset(algod_client, accounts[1]['pk'], asset_id)

Asset ID: 2
{
    "clawback": "WI6KG2FRR6FY7RSZH4AO4WDBY57R5IA74RX22EDQA4C6AOGVXAUAPUYWRU",
    "creator": "BOBMBO72PB2AM4RBSHB7B3ZZML2AW4SDIUFKJ3YLCPSDKNO3FOCRMN7H2A",
    "decimals": 0,
    "default-frozen": false,
    "freeze": "WI6KG2FRR6FY7RSZH4AO4WDBY57R5IA74RX22EDQA4C6AOGVXAUAPUYWRU",
    "manager": "BOBMBO72PB2AM4RBSHB7B3ZZML2AW4SDIUFKJ3YLCPSDKNO3FOCRMN7H2A",
    "name": "latinum",
    "name-b64": "bGF0aW51bQ==",
    "reserve": "WI6KG2FRR6FY7RSZH4AO4WDBY57R5IA74RX22EDQA4C6AOGVXAUAPUYWRU",
    "total": 1000,
    "unit-name": "LATINUM",
    "unit-name-b64": "TEFUSU5VTQ==",
    "url": "https://path/to/my/asset/details",
    "url-b64": "aHR0cHM6Ly9wYXRoL3RvL215L2Fzc2V0L2RldGFpbHM="
}


# Opt-in

In [25]:
# OPT-IN

# Check if asset_id is in account 3's asset holdings prior
# to opt-in
params = algod_client.suggested_params()
# comment these two lines if you want to use suggested params
# params.fee = 1000
# params.flat_fee = True

account_info = algod_client.account_info(accounts[3]['pk'])
holding = None
idx = 0
for my_account_info in account_info['assets']:
    scrutinized_asset = account_info['assets'][idx]
    idx = idx + 1    
    if (scrutinized_asset['asset-id'] == asset_id):
        holding = True
        break

if not holding:

    # Use the AssetTransferTxn class to transfer assets and opt-in
    txn = AssetTransferTxn(
        sender=accounts[3]['pk'],
        sp=params,
        receiver=accounts[3]["pk"],
        amt=0,
        index=asset_id)
    stxn = txn.sign(accounts[3]['sk'])
    # Send the transaction to the network and retrieve the txid.
    try:
        txid = algod_client.send_transaction(stxn)
        print("Signed transaction with txID: {}".format(txid))
        # Wait for the transaction to be confirmed
        confirmed_txn = wait_for_confirmation(algod_client, txid, 4) 
        print("TXID: ", txid)
        print("Result confirmed in round: {}".format(confirmed_txn['confirmed-round']))
 
    except Exception as err:
        print(err)
    # Now check the asset holding for that account.
    # This should now show a holding with a balance of 0.
    util.print_asset_holding(algod_client, accounts[3]['pk'], asset_id)


Signed transaction with txID: SOADSPOJMONXN44ZEK76JZ6UZSVFJNGLSZWROEHJOCKSYLVDLGJQ
TXID:  SOADSPOJMONXN44ZEK76JZ6UZSVFJNGLSZWROEHJOCKSYLVDLGJQ
Result confirmed in round: 4
Asset ID: 2
{
    "amount": 0,
    "asset-id": 2,
    "is-frozen": false
}


# Asset Transfer

In [26]:
# transfer asset of 10 from account 1 to account 3
params = algod_client.suggested_params()
# comment these two lines if you want to use suggested params
# params.fee = 1000
# params.flat_fee = True
txn = AssetTransferTxn(
    sender=accounts[1]['pk'],
    sp=params,
    receiver=accounts[3]["pk"],
    amt=10,
    index=asset_id)
stxn = txn.sign(accounts[1]['sk'])
# Send the transaction to the network and retrieve the txid.
try:
    txid = algod_client.send_transaction(stxn)
    print("Signed transaction with txID: {}".format(txid))
    # Wait for the transaction to be confirmed
    confirmed_txn = wait_for_confirmation(algod_client, txid, 4) 
    print("TXID: ", txid)
    print("Result confirmed in round: {}".format(confirmed_txn['confirmed-round']))

except Exception as err:
    print(err)
# The balance should now be 10.
util.print_asset_holding(algod_client, accounts[3]['pk'], asset_id)


Signed transaction with txID: 54YQ3DSNDMUJF66RFNPLVL4BGFYT4LCNVBX43P27GCRBQTQOBOXA
TXID:  54YQ3DSNDMUJF66RFNPLVL4BGFYT4LCNVBX43P27GCRBQTQOBOXA
Result confirmed in round: 5
Asset ID: 2
{
    "amount": 10,
    "asset-id": 2,
    "is-frozen": false
}


# Asset Freeze

In [27]:
params = algod_client.suggested_params()

# The freeze address (Account 2) freezes Account 3's latinum holdings.

txn = AssetFreezeTxn(
    sender=accounts[2]['pk'],
    sp=params,
    index=asset_id,
    target=accounts[3]["pk"],
    new_freeze_state=True   
    )
stxn = txn.sign(accounts[2]['sk'])
# Send the transaction to the network and retrieve the txid.
try:
    txid = algod_client.send_transaction(stxn)
    print("Signed transaction with txID: {}".format(txid))
    # Wait for the transaction to be confirmed
    confirmed_txn = wait_for_confirmation(algod_client, txid, 4)  
    print("TXID: ", txid)
    print("Result confirmed in round: {}".format(confirmed_txn['confirmed-round']))    
except Exception as err:
    print(err)
# The balance should now be 10 with frozen set to true.
util.print_asset_holding(algod_client, accounts[3]['pk'], asset_id)

Signed transaction with txID: FSSQPFYM5WEHAH2ESRL66WAQOXIIGO3UEBMYNMLVDWFEZQWBKKSQ
TXID:  FSSQPFYM5WEHAH2ESRL66WAQOXIIGO3UEBMYNMLVDWFEZQWBKKSQ
Result confirmed in round: 6
Asset ID: 2
{
    "amount": 10,
    "asset-id": 2,
    "is-frozen": true
}


# Revoke Asset

In [28]:
# The clawback address (Account 2) revokes 10 latinum from Account 3 and places it back with Account 1.
params = algod_client.suggested_params()
# comment these two lines if you want to use suggested params
# params.fee = 1000
# params.flat_fee = True

# Must be signed by the account that is the Asset's clawback address
txn = AssetTransferTxn(
    sender=accounts[2]['pk'],
    sp=params,
    receiver=accounts[1]["pk"],
    amt=10,
    index=asset_id,
    revocation_target=accounts[3]['pk']
    )
stxn = txn.sign(accounts[2]['sk'])
# Send the transaction to the network and retrieve the txid.
try:
    txid = algod_client.send_transaction(stxn)
    print("Signed transaction with txID: {}".format(txid))
    # Wait for the transaction to be confirmed
    confirmed_txn = wait_for_confirmation(algod_client, txid, 4)
    print("TXID: ", txid)
    print("Result confirmed in round: {}".format(confirmed_txn['confirmed-round']))      
except Exception as err:
    print(err)
# The balance of account 3 should now be 0.
# account_info = algod_client.account_info(accounts[3]['pk'])
print("Account 3")
util.print_asset_holding(algod_client, accounts[3]['pk'], asset_id)

# The balance of account 1 should increase by 10 to 1000.
print("Account 1")
util.print_asset_holding(algod_client, accounts[1]['pk'], asset_id)

Signed transaction with txID: XVPQNMS3SESH36WZ42YRGBJ6WOERRLLG2KHC3NNVFANWSWCZLNOQ
TXID:  XVPQNMS3SESH36WZ42YRGBJ6WOERRLLG2KHC3NNVFANWSWCZLNOQ
Result confirmed in round: 7
Account 3
Asset ID: 2
{
    "amount": 0,
    "asset-id": 2,
    "is-frozen": true
}
Account 1
Asset ID: 2
{
    "amount": 1000,
    "asset-id": 2,
    "is-frozen": false
}


# Destroy Asset

In [29]:
# With all assets back in the creator's account,
# the manager (Account 1) destroys the asset.
params = algod_client.suggested_params()
# comment these two lines if you want to use suggested params
# params.fee = 1000
# params.flat_fee = True

# Asset destroy transaction
txn = AssetConfigTxn(
    sender=accounts[1]['pk'],
    sp=params,
    index=asset_id,
    strict_empty_address_check=False
    )

# Sign with secret key of creator
stxn = txn.sign(accounts[1]['sk'])
# Send the transaction to the network and retrieve the txid.
# Send the transaction to the network and retrieve the txid.
try:
    txid = algod_client.send_transaction(stxn)
    print("Signed transaction with txID: {}".format(txid))
    # Wait for the transaction to be confirmed
    confirmed_txn = wait_for_confirmation(algod_client, txid, 4) 
    print("TXID: ", txid)
    print("Result confirmed in round: {}".format(confirmed_txn['confirmed-round']))     
except Exception as err:
    print(err)

# Asset was deleted.
try:
    print("Account 3 must do a transaction for an amount of 0, " )
    print("with a close_assets_to to the creator account, to clear it from its accountholdings")
    print("For Account 1, nothing should print after this as the asset is destroyed on the creator account")
   
    util.print_asset_holding(algod_client, accounts[1]['pk'], asset_id)
    util.print_created_asset(algod_client, accounts[1]['pk'], asset_id)
    # asset_info = algod_client.asset_info(asset_id)
except Exception as e:
    print(e)

Signed transaction with txID: BWWZE7VL2JPIIU4UJJRJLWGOEXROXDPQJ6NIPGXNZ7CZU3VCBWXA
TXID:  BWWZE7VL2JPIIU4UJJRJLWGOEXROXDPQJ6NIPGXNZ7CZU3VCBWXA
Result confirmed in round: 8
Account 3 must do a transaction for an amount of 0, 
with a close_assets_to to the creator account, to clear it from its accountholdings
For Account 1, nothing should print after this as the asset is destroyed on the creator account
